In [ ]:
# pip install spacy
# python -m spacy download en_core_web_sm
# note: i had to restart my computer after downloading the language package for it to work

In [112]:
import numpy as np
import pandas as pd
import spacy
import pyspark
from pyspark.ml.feature import HashingTF, IDF, StopWordsRemover
from pyspark.sql import SparkSession

In [114]:
print(StopWordsRemover)

<class 'pyspark.ml.feature.StopWordsRemover'>


In [2]:
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()

In [3]:
file = pd.read_csv("joined_data_v1.csv")

In [4]:
file = pd.DataFrame(file)
file.head()

,id,imdb_id,title,release_date,spoken_languages,genres1,genres2,genres3,genres4,genres5,...,director,poster_link,production_company,vote_average,vote_average (%),Internet_Movie_Database_rating,Rotten_Tomatoes_rating,Metacritic_rating,rating_average,writer
0,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
1,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
2,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
3,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
4,296524,tt1860357,Deepwater Horizon,9/29/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Action,NaN,NaN,NaN,...,Peter Berg,https://m.media-amazon.com/images/M/MV5BOTEzND...,Lionsgate,6.5,65%,71%,83%,68%,72%,"Matthew Michael Carnahan (screenplay by), Matt..."


In [5]:
file.drop_duplicates(keep='first', inplace=True)

In [6]:
file.head()

,id,imdb_id,title,release_date,spoken_languages,genres1,genres2,genres3,genres4,genres5,...,director,poster_link,production_company,vote_average,vote_average (%),Internet_Movie_Database_rating,Rotten_Tomatoes_rating,Metacritic_rating,rating_average,writer
0,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
2,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
4,296524,tt1860357,Deepwater Horizon,9/29/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Action,NaN,NaN,NaN,...,Peter Berg,https://m.media-amazon.com/images/M/MV5BOTEzND...,Lionsgate,6.5,65%,71%,83%,68%,72%,"Matthew Michael Carnahan (screenplay by), Matt..."
6,283366,tt1935859,Miss Peregrine's Home for Peculiar Children,9/28/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Fantasy,Adventure,NaN,NaN,...,Tim Burton,https://m.media-amazon.com/images/M/MV5BMTU0Nz...,20th Century Fox,6.5,65%,67%,65%,57%,64%,Ransom Riggs (based upon the novel written by)...
8,284564,tt3835080,31,9/15/2016,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Horror,Thriller,NaN,NaN,NaN,...,Rob Zombie,https://m.media-amazon.com/images/M/MV5BNTE0NT...,Bow and Arrow Entertainment,4.9,49%,51%,49%,35%,46%,Rob Zombie


In [7]:
# IGNORE THIS
#creating a list of the overview descriptions we want to run through spacy

# overview_list = file["overview"].tolist()
# overview_list

In [8]:
# IGNORE THIS
# overview1 = overview_list[0]
# overview1

In [9]:
# IGNORE THIS
# overview_df = file[["overview"]]
# overview_df

In [10]:
overview_df2 = file[["title", "overview"]].copy()
overview_df2.head()

,title,overview
0,The Birth of a Nation,"Nat Turner, a former slave in America, leads a..."
2,Sully,"On 15 January 2009, the world witnessed the 'M..."
4,Deepwater Horizon,A story set on the offshore drilling rig Deepw...
6,Miss Peregrine's Home for Peculiar Children,A teenager finds himself transported to an isl...
8,31,ROB ZOMBIE'S S P O O K H A U S 31


In [11]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [13]:
# IGNORE THIS
# from pyspark.sql.types import *
# mySchema = StructType([StructField("overview", StringType(), True)])

In [14]:
# IGNORE THIS
# df = spark.createDataFrame(overview_df,schema=mySchema)

In [15]:
sentenceData = spark.createDataFrame(overview_df2)
sentenceData.show() 

+--------------------+--------------------+
|               title|            overview|
+--------------------+--------------------+
|The Birth of a Na...|Nat Turner, a for...|
|               Sully|On 15 January 200...|
|   Deepwater Horizon|A story set on th...|
|Miss Peregrine's ...|A teenager finds ...|
|                  31|ROB ZOMBIE'S  S P...|
|             Snowden|CIA employee Edwa...|
|         Blair Witch|Students on a cam...|
|The Magnificent S...|Seven gun men in ...|
|              Morgan|A corporate risk-...|
|      The Dark Tower|The last Gunsling...|
|          Nine Lives|A stuffy business...|
|       Don't Breathe|A group of teens ...|
|       Suicide Squad|From DC Comics co...|
|Kubo and the Two ...|Kubo mesmerizes t...|
|             Ben-Hur|A falsely accused...|
|            War Dogs|Based on the true...|
|  Hell or High Water|A divorced dad an...|
|          Anthropoid|In December 1941,...|
|       Pete's Dragon|Pete is a mysteri...|
|            Rock Dog|When a rad

In [16]:
tokenizer = Tokenizer(inputCol="overview", outputCol="words")
tokenizer

Tokenizer_a3fe50f9a80c

In [17]:
# Transform and show DataFrame
tokenized = tokenizer.transform(sentenceData)
tokenized.show(truncate=False)

+-------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title                                      |overview               

In [18]:
# Instantiate Remover
remover = StopWordsRemover(inputCol="words", outputCol="stopwords")

In [19]:
# Transform and show data
tokenized = remover.transform(tokenized)
tokenized.show(truncate=False)

+-------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------

# Spacy NLP

In [119]:
# Load English tokenizer, tagger, parser, NER and word vectors

nlp = spacy.load("en_core_web_sm")

In [21]:
#change length to 193 to get all movies
length = 10
i = 0

#empty dictionary
#nested_dict = {}

while i < length: 
    title = str(tokenized.select("title").collect()[0+i])
    clean_title = title[10:]
    text1 =str(tokenized.select("stopwords").collect()[0+i])
    clean_text = text1[16:]
    docs=nlp(clean_text)
    
# populating dictionary attempt 1 (does not work)
    # nested_dict[clean_title] = {"Nouns":[chunk.text for chunk in docs.noun_chunks], "Verbs":[token.lemma_ for token in docs if token.pos_ == "VERB"], "Adverbs": [token.lemma_ for token in docs if token.pos_ == "ADV"], "Adjectives:", [token.lemma_ for token in docs if token.pos_ == "ADJ"]}

# populating dictionary attempt 2 (also does not work)
#     nested_dict[clean_title]['Nouns'] = [chunk.text for chunk in docs.noun_chunks]
#     nested_dict[clean_title]['Verbs'] = [token.lemma_ for token in docs if token.pos_ == "VERB"]
#     nested_dict[clean_title]['Adverbs'] = [token.lemma_ for token in docs if token.pos_ == "ADV"] 
#     nested_dict[clean_title]['Adjectives'] = [token.lemma_ for token in docs if token.pos_ == "ADJ"]
    
#    print(nested_dict)

    print("Movie_Title:", clean_title)
    print("Nouns:", [chunk.text for chunk in docs.noun_chunks])
    print("Verbs:", [token.lemma_ for token in docs if token.pos_ == "VERB"])
    print("Adverbs:", [token.lemma_ for token in docs if token.pos_ == "ADV"])
    print("Adjectives:", [token.lemma_ for token in docs if token.pos_ == "ADJ"])
    
    print("")
    
    i += 1

Movie_Title: 'The Birth of a Nation')
Nouns: ["'former', 'slave', 'america", "'liberation", ", 'movement", ", 'african-americans", 'virgina', ", 'results", "'retaliation"]
Verbs: ['lead']
Adverbs: ['nat']
Adjectives: ['former', 'free', 'african', 'violent']

Movie_Title: 'Sully')
Nouns: ['15\', \'january\', \'2009,\', \'world\', \'witnessed\', "\'miracle', '"hudson', "'captain", 'sullenberger', "'plane", "'waters", "'hudson', 'river", "'lives", "'reputation", 'career']
Verbs: ['witness', 'glide', 'save', 'herald', 'unfold', 'threaten', 'destroy']
Adverbs: ['sully', 'however', 'even', 'sully']
Adjectives: ['january', 'disabled', 'frigid', 'public', 'unprecedented']

Movie_Title: 'Deepwater Horizon')
Nouns: ['story', ", 'drilling", ", 'rig", "'deepwater", "'horizon", "'spill"]
Verbs: ['explode', 'create']
Adverbs: []
Adjectives: ['offshore', 'bad']

Movie_Title: "Miss Peregrine's Home for Peculiar Children")
Nouns: ['teenager', "'island", "'must', 'help", "'protect", "'group", "'orphans"

In [22]:
tokenized.show()

+--------------------+--------------------+--------------------+--------------------+
|               title|            overview|               words|           stopwords|
+--------------------+--------------------+--------------------+--------------------+
|The Birth of a Na...|Nat Turner, a for...|[nat, turner,, a,...|[nat, turner,, fo...|
|               Sully|On 15 January 200...|[on, 15, january,...|[15, january, 200...|
|   Deepwater Horizon|A story set on th...|[a, story, set, o...|[story, set, offs...|
|Miss Peregrine's ...|A teenager finds ...|[a, teenager, fin...|[teenager, finds,...|
|                  31|ROB ZOMBIE'S  S P...|[rob, zombie's, ,...|[rob, zombie's, ,...|
|             Snowden|CIA employee Edwa...|[cia, employee, e...|[cia, employee, e...|
|         Blair Witch|Students on a cam...|[students, on, a,...|[students, campin...|
|The Magnificent S...|Seven gun men in ...|[seven, gun, men,...|[seven, gun, men,...|
|              Morgan|A corporate risk-...|[a, corpora

# Thesaurus API

In [23]:
import requests
import json
api_key = "bac73abe-335b-4c95-b989-bd87a4b4751b"
# https://www.dictionaryapi.com/api/v3/references/thesaurus/json/umpire?key=your-api-key

In [24]:
tokenized.select("stopwords").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|stopwords                                                                                                                                                                                                                                                                                                                                       |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [25]:
import pyspark.sql.functions as f
# tokenized.select("stopwords").toList
# tokenized.withColumn("first_two", f.array([f.col("stopwords")[0], f.col("stopwords")[1]])).show()
# tokenized.col("stopwords")[0]

AttributeError: 'DataFrame' object has no attribute 'col'

In [56]:

# tokenized.select("stopwords").map(r => r.getString(0)).collect.toList
# Products_list = tokenized.select("stopwords").tolist()
# print (Products_list)
data_array =  np.array(tokenized.select("stopwords").collect())
data_array = data_array[0:2]  #for testing taking 10 value for API call

In [57]:
data_array

array([[list(['nat', 'turner,', 'former', 'slave', 'america,', 'leads', 'liberation', 'movement', '1831', 'free', 'african-americans', 'virgina', 'results', 'violent', 'retaliation', 'whites.'])],
       [list(['15', 'january', '2009,', 'world', 'witnessed', "'miracle", "hudson'", 'captain', "'sully'", 'sullenberger', 'glided', 'disabled', 'plane', 'onto', 'frigid', 'waters', 'hudson', 'river,', 'saving', 'lives', '155', 'aboard.', 'however,', 'even', 'sully', 'heralded', 'public', 'media', 'unprecedented', 'feat', 'aviation', 'skill,', 'investigation', 'unfolding', 'threatened', 'destroy', 'reputation', 'career.'])]],
      dtype=object)

In [71]:
print(data_array[0][0])

['nat', 'turner,', 'former', 'slave', 'america,', 'leads', 'liberation', 'movement', '1831', 'free', 'african-americans', 'virgina', 'results', 'violent', 'retaliation', 'whites.']


In [67]:
synonyms = []
i=0
j=0
k=0
for i in range(len(data_array)):
    for j in range(len(data_array[i])):
        for k in range(len(data_array[i][j])):
            word = data_array[i][j][k]
#             print(word)
            if len(word)>2:
                target_url = 'https://www.dictionaryapi.com/api/v3/references/thesaurus/json/' + word + '?key=' + api_key
                data = requests.get(target_url).json()
#                 print(data)
            if data:
#                 synonyms.append(data[0]["meta"]["syns"])

SyntaxError: unexpected EOF while parsing (<ipython-input-67-8398d3f51769>, line 15)

In [48]:
synonyms

[]

In [49]:
data

['neat',
 'bat',
 'nap',
 'cat',
 'nay',
 'eat',
 'natch',
 'fat',
 'neath',
 'hat',
 'nays',
 'kat',
 'note',
 'mat',
 'net',
 'nab',
 'nit',
 'nag',
 'nut',
 'pat']

In [72]:
words = data_array[0][0]
print(words)

['nat', 'turner,', 'former', 'slave', 'america,', 'leads', 'liberation', 'movement', '1831', 'free', 'african-americans', 'virgina', 'results', 'violent', 'retaliation', 'whites.']


In [108]:
for word in words:
    if len(word)>2:
                target_url = 'https://www.dictionaryapi.com/api/v3/references/thesaurus/json/' + word + '?key=' + api_key
                data = requests.get(target_url).json()
                print(data[0]["meta"]["syns"])
                data=""
#         if len(data)>0:
#                 synonyms = data[0]["meta"]["syns"]

TypeError: string indices must be integers

In [109]:

word = 'former'
target_url = 'https://www.dictionaryapi.com/api/v3/references/thesaurus/json/' + word + '?key=' + api_key
data = requests.get(target_url).json()
data[0]["meta"]["syns"]

[['erstwhile',
  'late',
  'old',
  'once',
  'onetime',
  'other',
  'past',
  'quondam',
  'sometime',
  'whilom'],
 ['antecedent',
  'anterior',
  'foregoing',
  'precedent',
  'preceding',
  'previous',
  'prior']]

In [107]:
data[0]["meta"]["syns"]

[['erstwhile',
  'late',
  'old',
  'once',
  'onetime',
  'other',
  'past',
  'quondam',
  'sometime',
  'whilom'],
 ['antecedent',
  'anterior',
  'foregoing',
  'precedent',
  'preceding',
  'previous',
  'prior']]

In [74]:
synonyms.append(data[0]["meta"]["syns"])

In [81]:
synonyms[0][1]

['antecedent',
 'anterior',
 'foregoing',
 'precedent',
 'preceding',
 'previous',
 'prior']

In [100]:
tokenized.show()

+--------------------+--------------------+--------------------+--------------------+
|               title|            overview|               words|           stopwords|
+--------------------+--------------------+--------------------+--------------------+
|The Birth of a Na...|Nat Turner, a for...|[nat, turner,, a,...|[nat, turner,, fo...|
|               Sully|On 15 January 200...|[on, 15, january,...|[15, january, 200...|
|   Deepwater Horizon|A story set on th...|[a, story, set, o...|[story, set, offs...|
|Miss Peregrine's ...|A teenager finds ...|[a, teenager, fin...|[teenager, finds,...|
|                  31|ROB ZOMBIE'S  S P...|[rob, zombie's, ,...|[rob, zombie's, ,...|
|             Snowden|CIA employee Edwa...|[cia, employee, e...|[cia, employee, e...|
|         Blair Witch|Students on a cam...|[students, on, a,...|[students, campin...|
|The Magnificent S...|Seven gun men in ...|[seven, gun, men,...|[seven, gun, men,...|
|              Morgan|A corporate risk-...|[a, corpora

In [115]:
# Run the hashing term frequency
hashing = HashingTF(inputCol="stopwords", outputCol="hashedValues", numFeatures=pow(2,8))

# Transform into a DF
hashed_df = hashing.transform(tokenized)
# Display new DataFrame
hashed_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|            overview|               words|           stopwords|        hashedValues|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|The Birth of a Na...|Nat Turner, a for...|[nat, turner,, a,...|[nat, turner,, fo...|(256,[8,30,42,43,...|
|               Sully|On 15 January 200...|[on, 15, january,...|[15, january, 200...|(256,[4,8,11,29,4...|
|   Deepwater Horizon|A story set on th...|[a, story, set, o...|[story, set, offs...|(256,[0,13,24,27,...|
|Miss Peregrine's ...|A teenager finds ...|[a, teenager, fin...|[teenager, finds,...|(256,[9,25,28,31,...|
|                  31|ROB ZOMBIE'S  S P...|[rob, zombie's, ,...|[rob, zombie's, ,...|(256,[20,92,96,11...|
|             Snowden|CIA employee Edwa...|[cia, employee, e...|[cia, employee, e...|(256,[22,52,61,10...|
|         Blair Witch|Students on a c

In [116]:
# Fit the IDF on the data set 
idf = IDF(inputCol="hashedValues", outputCol="features")
idfModel = idf.fit(hashed_df)
rescaledData = idfModel.transform(hashed_df)

In [117]:
# Display the DataFrame
rescaledData.select("title", "features").show(truncate=False)

+-------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title                                      |features                                                                                                                           

In [ ]:
inputword = "Star wars"

In [110]:
import spacy

In [ ]:
nlp = spacy.load('en_core_web_md')

In [111]:
entity_extractor = TextMatcher() \
    .setInputCols(["inputCol"])\
    .setOutputCol("entity")\
    .setEntities(tokenized.select("stopwords"))

NameError: name 'TextMatcher' is not defined

In [122]:
oranges = nlp(u"apples and oranges")
apples = nlp(u"You saw me")
apples_oranges = apples.similarity(oranges)
oranges_apples = oranges.similarity(apples)
assert apples_oranges == oranges_apples

/Users/ericH/anaconda3/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/Users/ericH/anaconda3/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use